# **Cinética Enzimática II**

## Las celdas inferiores pueden ejecutarse con normalidad para que funcione correctamente el cuadernillo (la sección de conversión a PDF fue localizada al final del cuadernillo)

In [ ]:
%%capture out
# Instala extensiones para widgets interactivos
!pip install jupyter_contrib_nbextensions
!jupyter contrib nbextension install
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
# Modulos necesarios para ejecutar el código

import os                             # Modulo para operaciones a nivel de Sistema Operativo
import sys                            # Operaciones a nivel de sistema (para añadir rutas)
import math                           # Operaciones matemáticas 
import numpy as np                    # Creación y manejo de arreglos, además de operaciones con estos
import pandas as pd                   # Manejo y operaciones con bases de datos grandes
import matplotlib                     # Creación de gráficos
import matplotlib.pyplot as plt       # Módulo específico de gráficos
import warnings                       # Utilizado para silenciar ciertas advertencias
import csv                            # Importar, exportar, leer y escribir archivos .csv
import scipy.stats                    # Módulo científico estadístico
import nbconvert                      # Para convertir cuadernillo a PDF
import ipywidgets as widgets          # Utilizado para crear outputs interactivos
import IPython                        # Utilizado para el manejo del output
from IPython.display import HTML      # Renderiza HTML en el output

# Disminuye la verbosidad de matplotlib al crear gráficos
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')

# Utilizado para dar distintos estilos a matplotlib
# y que los gráficos se muestren en línea con el código
plt.style.use('ggplot')
%matplotlib inline

In [ ]:
# Para permitir el acceso a archivos de google drive
from google.colab import drive
drive.mount('/content/drive')
# Imprime el directorio en el que se encuentra
os.getcwd()

In [ ]:
# Se cambia de directorio al de drives compartidos
%cd '/content/drive/Shared drives'

In [ ]:
# Se lista el contenido del directorio
!ls
# Recordar que se debe seleccionar la carpeta suya!

In [ ]:
# Esta celda añade la ruta especificada a las rutas de busqueda del sistema,
# esto permite que se puedan importar módulos externos a python
sys.path.append('/content/drive/Shared\ drives/Cinética - Rodriguez Prieto Jose Ignacio')

# Esta celda no va a retornar nada

In [ ]:
# Aquí cambiamos de directorio (con el cual trabajaremos)
%cd "/content/drive/Shared drives/Cinética - Rodriguez Prieto Jose Ignacio"
#%cd "/content/drive/Shared drives/PEGUE AQUI EL NOMBRE DE SU CARPETA"

In [ ]:
# Finalmente acá se importa un módulo diseñado para este curso
import cinetica_enzimatica_estudiantes_bio266 as ce

# **Obtención de la curva de saturación para la β-galactosidasa**

**Diseño Experimental**:

Se determinará la velocidad inicial de la enzima (μM/min) a concentraciones de sustrato crecientes (en μM). Esto permite obtener los parámetros cinéticos de la enzima (Km y Vmáx). 
Escoger una dilución de enzima apropiada según el resultado del práctico anterior. Preparar 9 diluciones de sustrato oNPGal 3 mg/ml, además del blanco sin sustrato. Incubar por el tiempo determinado en el práctico anterior a 37ºC. Detener la reacción con 0,5 ml de Na2CO3 1M pH 10. Se determina la concentración de producto a una longitud de onda de 420 nm.

Incubar la mezcla a 37°C y detener la reacción al tiempo determinado con 500 μl de Na2CO3 1 M pH 10.
Grafique los valores de velocidad inicial vs concentración de sustrato (gráfico de Michaelis Menten) y los valores recíprocos 1/S (μM) vs 1/Velocidad (μM de producto/min) (gráfico de dobles recíprocos o Lineweaver y Burk).


In [ ]:
# Curva de Saturación (Gráfico de Michaelis Menten)

# Primero debemos abrir el archivo, para lo cual se cargan los datos
# utlizando las rutas de archivos (si se conocen los nombres), o
# la barra izquierda de Google Colab, que permite navegar el Drive
# hasta encontrar los archivos
ruta_de_archivo = "micment_data_problem_data.csv"

# Esta línea lee el archivo utilizando Pandas,
# dando como resultado un DataFrame
datos_curvasat = pd.read_csv(ruta_de_archivo)

%load_ext google.colab.data_table
datos_curvasat
#print(datos)

In [ ]:
# Se crea la figura y un objeto ax
fig, ax = plt.subplots(figsize=(10,5), dpi=150)

# Se realiza un bucle para graficar todos los datos en una misma figura
# Los objetos DataFrame que crea Pandas al importar un objeto se integran
# facilmente con matplotlib, por lo que se facilita la creación de gráficos
datos_curvasat.plot(
    datos_curvasat.columns[0], 
    datos_curvasat.columns[1], 
    kind='scatter',
    color="b",
    alpha=0.9,
    s=2, 
    ax=ax,
    label=datos_curvasat.columns[1])

# Título y nombres de ejes (deben modificarse de acuerdo a lo graficado)
plt.title("Título del Gráfico")     # Modificar de acuerdo a lo graficado
plt.ylabel("Nombre de eje Y")       # Modificar de acuerdo a lo graficado
plt.xlabel("Nombre de eje X")       # Modificar de acuerdo a lo graficado

# Opciones para ajustar la distribución de objetos en el gráfico
# No modificar a menos que desee experimentar!
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.tight_layout()

**Recuerde que para este ajuste en particular debe ingresar la $V_{lim}$ ($V_{max}$ ) en $\mu$M/seg (para esto, divida el límite inferior y superior para $V_{max}$  por 60)**

In [ ]:
# Ajustar curvas

# Datos deben ser transformados a arreglos numpy para ser ajustados
# IMPORTANTE: Para cambiar de columna se debe cambiar el índice de datos_y
datos_x_curva_sat = datos_curvasat[datos_curvasat.columns[0]].to_numpy()
datos_y_curva_sat = datos_curvasat[datos_curvasat.columns[1]].to_numpy()

# Se ajustan los datos experimentales a los predichos por el modelo
# para mayor informacion acerca del modelo utilizado refierase a las clases de cinética enzimática
resultados_ajuste_curva_sat = ce.ajustar_modelo(ce.fit_micment_dil, datos_x_curva_sat, datos_y_curva_sat)

# resultados_ajuste es un diccionario que tiene dos llaves:
# "prediccionesModelo": la cual corresponde a un arreglo de datos generados por el modelo
# "difDatosModelo": la cual corresponde a un arreglo de datos y se utilizará para graficar los residuales

In [ ]:
# Veamos qué tan bien se ajusta el modelo a los datos experimentales
fig, ax = plt.subplots(figsize=(10,5), dpi=150)

# Primero se graficarán los datos experimentales
datos_curvasat.plot(
        datos_curvasat.columns[0], 
        datos_curvasat.columns[1], 
        kind='scatter',
        color="r",
        alpha=0.4,
        s=4, 
        ax=ax,
        label=datos_curvasat.columns[1])

# Luego se graficarán los datos obtenidos mediante el modelo
ax.plot(
    datos_x_curva_sat, 
    resultados_ajuste_curva_sat["prediccionesModelo"], 
    color="b", 
    alpha=0.8,
    lw=2.0, 
    ls="--", 
    label="Ajuste")

plt.legend()
plt.title("Título del Gráfico")     # Modificar de acuerdo a lo graficado
plt.ylabel("Nombre de eje Y")       # Modificar de acuerdo a lo graficado
plt.xlabel("Nombre de eje X")       # Modificar de acuerdo a lo graficado

## **Gráficos y Ajuste a Lineweaver-Burk**

Para realizar los gráficos y ajustes a Lineweaver-Burk los datos experimentales se invierten ($1/datos$)

In [ ]:
# Se crea la figura y un objeto ax
fig, ax = plt.subplots(figsize=(10,5), dpi=150)

# Se grafican los datos
plt.scatter(
    np.divide(1, datos_curvasat[datos_curvasat.columns[0]][100:]), 
    np.divide(1, datos_curvasat[datos_curvasat.columns[1]][100:]),
    color="b",
    alpha=0.8,
    s=4,
    label=datos_curvasat.columns[1])

plt.title("Título del Gráfico")     # Modificar de acuerdo a lo graficado
plt.ylabel("Nombre de eje Y")       # Modificar de acuerdo a lo graficado
plt.xlabel("Nombre de eje X")       # Modificar de acuerdo a lo graficado

# Opciones para ajustar la distribución de objetos en el gráfico
# No modificar a menos que desee experimentar!
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.tight_layout()

### **Importante**: Recuerde que debido a que la pendiente de esta recta corresponde a $\frac{K_m}{V_{max}}$ y que $V_{max}$ debe ser ajustada utilizando unidades de $\frac{\mu M}{s}$, el valor de la pendiente incrementa sustancialmente con valores pequeños de $V_{max}$. A modo de ejemplo, consideremos que $V_{max} \approx 0.1 \frac{\mu M}{s}$ y $K_m \approx 200 \mu M$. Al calcular $\frac{K_m}{V_{max}}$ se obtiene un valor de **2000**. Considerando esto, recuerde probar con distintos límites inferiores y superiores este ajuste.

In [ ]:
# Ajustar curvas

# Datos deben ser transformados a arreglos numpy para ser ajustados
datos_x_lwburk = np.divide(1, datos_x_curva_sat[100:])
datos_y_lwburk = np.divide(1, datos_y_curva_sat[100:])

# Se ajustan los datos experimentales a los predichos por el modelo
# para mayor informacion acerca del modelo utilizado refierase a las clases de cinética enzimática
resultados_ajuste_lwburk = ce.ajustar_modelo(ce.fit_lwburk, datos_x_lwburk, datos_y_lwburk)

# resultados_ajuste es un diccionario que tiene dos llaves:
# "prediccionesModelo": la cual corresponde a un arreglo de datos generados por el modelo
# "difDatosModelo": la cual corresponde a un arreglo de datos y se utilizará para graficar los residuales

In [ ]:
# Se crea la figura y un objeto ax
fig, ax = plt.subplots(figsize=(10,5), dpi=150)

# Se grafican los datos transformados a Lineweaver-Burk
plt.scatter(
    datos_x_lwburk, 
    datos_y_lwburk,
    color="r",
    alpha=0.8,
    s=4,
    label="Datos experimentales")

# Se grafica el ajuste obtenido
ax.plot(
    datos_x_lwburk, 
    resultados_ajuste_lwburk["prediccionesModelo"], 
    color="b", 
    alpha=0.8,
    lw=2.0, 
    ls="--", 
    label="Ajuste a Lineweaver-Burk")

plt.title("Título del Gráfico")     # Modificar de acuerdo a lo graficado
plt.ylabel("Nombre de eje Y")       # Modificar de acuerdo a lo graficado
plt.xlabel("Nombre de eje X")       # Modificar de acuerdo a lo graficado

# Opciones para ajustar la distribución de objetos en el gráfico
# No modificar a menos que desee experimentar!
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.tight_layout()

In [ ]:
# Esta funcion permite calcular el intercepto del eje X
ce.find_x_intercept()

## **Preguntas a desarrollar**

Complete la siguiente tabla con los valores de $V_{max}$ y $K_m$ obtenidos en los ajustes previos.

Haz doble clic (o ingresa) para editar

|  | $V_{max}$ | $K_m$ |
|:-:|:-:|:-:|
| **Michaelis-<br>Menten** | Inserte su valor | Inserte su valor |
| **Lineweaver-<br>Burk** | Inserte su valor | Inserte su valor |

**Compare los parametros ajustados a la curva de Michaelis-Menten y a la recta de Lineweaver-Burk ¿En qué casos utilizaría un método sobre otro? ¿Existen otros gráficos que se utilicen para determinar los parámetros cinéticos?**

# **Efecto de inhibidores sobre la actividad β-galactosidasa**

**Diseño Experimental**:

Se estudiará el efecto inhibitorio de los compuestos galactosa y glucosa en forma independiente. Para esto se determina la velocidad inicial en función de las mismas concentraciones crecientes del sustrato oNPGal anteriores, en presencia de dos concentraciones de galactosa en un caso, y en presencia de dos concentraciones de glucosa en el otro caso.

Se grafica velocidad (μM de producto/min) en función de la concentración de sustrato (μM) en presencia de cada uno de los inhibidores. Por lo tanto, se realizan cuatro curvas con concentraciones crecientes de sustrato. Luego se grafican las curvas de dobles recíprocos 1/S (μM) v/s 1/Velocidad (μM de producto/min) sin inhibidor y con cada inhibidor en dos concentraciones cada uno.

A partir de estos gráficos se determina como varían Km y Vmáx, y se define así el tipo de inhibición que ejercen galactosa y glucosa.



In [ ]:
# Gráfico interactivo inhibidor competitivo (Curva de Michaelis-Menten)
ce.interactive_comp_inhibitor_graph1(np.linspace(0, 10000, 10000//10), ce.fit_micment_comp_inhibitor)

In [ ]:
# Gráfico interactivo inhibidor competitivo (Lineweaver-Burk)
ce.interactive_comp_inhibitor_graph2(np.linspace(1000, 10000, 10000//10), ce.fit_micment_comp_inhibitor)

## **Gráficos y Ajustes a curva de Michaelis-Menten**

In [ ]:
# En primer lugar se debe leer el archivo que contiene los datos
ruta_de_archivo = "comp_inhibitor_data_problem_data.csv"
datos_inhibcomp = pd.read_csv(ruta_de_archivo)

datos_inhibcomp
#print(datos)

In [ ]:
colores = plt.cm.inferno_r(np.linspace(0.25, 0.75, len(list(datos_inhibcomp.columns))-1))

# Se crea la figura y un objeto ax
fig, ax = plt.subplots(figsize=(10,5), dpi=150)

# Se realiza un bucle para graficar todos los datos en una misma figura
for i in range(1, len(list(datos_inhibcomp.columns))):
    datos_inhibcomp.plot(
        datos_inhibcomp.columns[0], 
        datos_inhibcomp.columns[i], 
        kind='scatter',
        color=colores[i-1],
        alpha=0.8,
        s=4, 
        ax=ax,
        label=datos_inhibcomp.columns[i])

# Además, para tener una mayor claridad acerca del efecto que tiene este tipo de
# inhibidores, se grafica la curva de Michaelis-Menten sin inhibidor   
ax.plot(
    datos_x_curva_sat, 
    resultados_ajuste_curva_sat["prediccionesModelo"], 
    color="g", 
    alpha=0.8,
    lw=2.0, 
    ls="--", 
    label="Ajuste a MM sin inhibidores")

plt.title("Título del Gráfico")     # Modificar de acuerdo a lo graficado
plt.ylabel("Nombre de eje Y")       # Modificar de acuerdo a lo graficado
plt.xlabel("Nombre de eje X")       # Modificar de acuerdo a lo graficado

# Opciones para ajustar la distribución de objetos en el gráfico
# No modificar a menos que desee experimentar!
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.tight_layout()

In [ ]:
# Ajustar curvas

# Datos deben ser transformados a arreglos numpy para ser ajustados
# IMPORTANTE: Para cambiar de columna se debe cambiar el índice de la columna datos_y
# (donde dice datos_inhibcomp.columns[indice])
datos_x_inhibcomp = datos_inhibcomp[datos_inhibcomp.columns[0]].to_numpy()
datos_y_inhibcomp = datos_inhibcomp[datos_inhibcomp.columns[2]].to_numpy()

# Se ajustan los datos experimentales a los predichos por el modelo
resultados_ajuste = ce.ajustar_modelo(ce.fit_micment_comp_inhibitor, datos_x_inhibcomp, datos_y_inhibcomp)

# resultados_ajuste es un diccionario que tiene dos llaves:
# "prediccionesModelo": la cual corresponde a un arreglo de datos generados por el modelo
# "difDatosModelo": la cual corresponde a un arreglo de datos y se utilizará para graficar los residuales

In [ ]:
# Veamos qué tan bien se ajusta el modelo a los datos experimentales
fig, ax = plt.subplots(figsize=(10,5), dpi=150)

# Primero se graficarán los datos experimentales
datos_inhibcomp.plot(
        datos_inhibcomp.columns[0], 
        datos_inhibcomp.columns[2], 
        kind='scatter',
        color="r",
        alpha=0.4,
        s=4, 
        ax=ax,
        label=datos_inhibcomp.columns[2])

# Luego se graficarán los datos obtenidos mediante el modelo
ax.plot(
    datos_x_inhibcomp, 
    resultados_ajuste["prediccionesModelo"], 
    color="b", 
    alpha=1.0,
    lw=2.0, 
    ls="--", 
    label="Ajuste")

# Se grafica ademas el ajuste a la curva de MM sin inhibidores
ax.plot(
    datos_x_curva_sat, 
    resultados_ajuste_curva_sat["prediccionesModelo"], 
    color="g", 
    alpha=0.8,
    lw=2.0, 
    ls="--", 
    label="Ajuste a MM sin inhibidores")

plt.legend()
plt.title("Título del Gráfico")     # Modificar de acuerdo a lo graficado
plt.ylabel("Nombre de eje Y")       # Modificar de acuerdo a lo graficado
plt.xlabel("Nombre de eje X")       # Modificar de acuerdo a lo graficado

## **Gráficos y ajustes a Lineweaver-Burk**

In [ ]:
# Se crea la figura y un objeto ax
fig, ax = plt.subplots(figsize=(10,5), dpi=150)

# Para graficar los inversos, se transforman los datos que ya se cargaron al cuadernillo
# Para esto se utiliza la funcion np.divide()
# IMPORTANTE: Recuerde modificar el índice de su columna (a la que usted elija)
plt.scatter(
    np.divide(1, datos_inhibcomp[datos_inhibcomp.columns[0]][300:]), 
    np.divide(1, datos_inhibcomp[datos_inhibcomp.columns[2]][300:]),
    color="b",
    alpha=0.8,
    s=4,
    label=datos_inhibcomp.columns[2])

plt.title("Título del Gráfico")     # Modificar de acuerdo a lo graficado
plt.ylabel("Nombre de eje Y")       # Modificar de acuerdo a lo graficado
plt.xlabel("Nombre de eje X")       # Modificar de acuerdo a lo graficado

# Opciones para ajustar la distribución de objetos en el gráfico
# No modificar a menos que desee experimentar!
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.tight_layout()

In [ ]:
# Ajustar curvas

# Datos deben ser transformados a arreglos numpy para ser ajustados
# IMPORTANTE: Para cambiar de columna se debe cambiar el índice de la columna datos_y
# (donde dice datos_inhibcomp.columns[indice])
datos_x_lwburk = np.divide(1, datos_inhibcomp[datos_inhibcomp.columns[0]][300:])
datos_y_lwburk = np.divide(1, datos_inhibcomp[datos_inhibcomp.columns[2]][300:])

# Se ajustan los datos experimentales a los predichos por el modelo
# para mayor informacion acerca del modelo utilizado refierase a las clases de cinética enzimática
resultados_ajuste_lwburk = ce.ajustar_modelo(ce.fit_lwburk, datos_x_lwburk, datos_y_lwburk)

# resultados_ajuste es un diccionario que tiene dos llaves:
# "prediccionesModelo": la cual corresponde a un arreglo de datos generados por el modelo
# "difDatosModelo": la cual corresponde a un arreglo de datos y se utilizará para graficar los residuales

In [ ]:
# Se crea la figura y un objeto ax
fig, ax = plt.subplots(figsize=(10,5), dpi=150)

# Se grafican los datos transformados a Lineweaver-Burk 
plt.scatter(
    datos_x_lwburk, 
    datos_y_lwburk,
    color="r",
    alpha=0.8,
    s=4,
    label="Datos experimentales")

# Y además se grafica la recta obtenida luego de ajustar los parámetros
ax.plot(
    datos_x_lwburk, 
    resultados_ajuste_lwburk["prediccionesModelo"], 
    color="b", 
    alpha=0.8,
    lw=2.0, 
    ls="--", 
    label="Ajuste a Lineweaver-Burk")

plt.title("Título del Gráfico")     # Modificar de acuerdo a lo graficado
plt.ylabel("Nombre de eje Y")       # Modificar de acuerdo a lo graficado
plt.xlabel("Nombre de eje X")       # Modificar de acuerdo a lo graficado

# Opciones para ajustar la distribución de objetos en el gráfico
# No modificar a menos que desee experimentar!
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.tight_layout()

In [ ]:
# Esta funcion permite calcular el intercepto del eje X
ce.find_x_intercept()

## **Preguntas a desarrollar**

Complete la siguiente tabla con los valores de $V_{max}$ y $K_m$ que obtuvo en los ajustes realizados de esta sección. Recuerde que:

- Los parámetros ajustados sin inhibidor se obtuvieron en la primera sección (Curva de Saturación)

- Para obtener los parámetros a partir de Lineweaver-Burk deberá realizar cálculos (ya que no se obtienen directamente desde el ajuste a la recta)

Haz doble clic (o ingresa) para editar

|  | $V_{max}$ | $K_m$ |
|:-:|:-:|:-:|
| **Michaelis-<br>Menten** | -------- | --------- |
| Sin Inhibidor | Su valor acá | Su valor acá |
| Con Inhibidor | Su valor acá | Su valor acá |
| **Lineweaver-<br>Burk** | -------- | --------- |
| Sin Inhibidor | Su valor acá | Su valor acá |
| Con Inhibidor | Su valor acá | Su valor acá |

**En base a la tabla superior ¿Qué parámetro(s) se ve(n) afectado(s)? ¿Cómo actúan los inhibidores competitivos a nivel biológico?** (Hint: Considere efectos en la Enzima y/o complejo Enzima/Sustrato)

**¿Qué es la $K_{ic}$? ¿Qué efecto tendría el aumentar o disminuir su valor?**

**Al realizar un ensayo enzimático la tasa de reacción observada es *menor* a la esperada. Usted sabe además que su reacción requiere de un determinado compuesto, sin embargo este ha sido descrito como un posible inhibidor competitivo de la enzima. ¿Cómo podría aumentar la tasa de reacción sin eliminar este compuesto?**

# **Efecto de un inhibidor mixto**


In [ ]:
# Gráfico Inhibidores No-Competitivos
# Recomiendo no cambiar parametros de esta funcion
ce.interactive_mixed_inhibitor_graph1(np.linspace(0, 10000, 10000//10), ce.fit_micment_mixed_inhibitor)

In [ ]:
# Gráfico Inhibidores No-Competitivos
# Recomiendo no cambiar parametros de esta funcion
ce.interactive_mixed_inhibitor_graph2(np.linspace(1000, 10000, 10000//10), ce.fit_micment_mixed_inhibitor)

In [ ]:
# En primer lugar se debe leer el archivo que contiene los datos
ruta_de_archivo = "incognito.csv"
datos_inhibnocomp = pd.read_csv(ruta_de_archivo)

datos_inhibnocomp
#print(datos)

In [ ]:
colores = plt.cm.viridis_r(np.linspace(0.25, 0.75, len(list(datos_inhibnocomp.columns))-1))

# Se crea la figura y un objeto ax (recuerde que el objeto ax será donde se grafiquen las curvas)
fig, ax = plt.subplots(figsize=(10,5), dpi=150)

# Se realiza un bucle para graficar todos los datos en una misma figura
for i in range(1, len(list(datos_inhibnocomp.columns))):
    datos_inhibnocomp.plot(
        datos_inhibnocomp.columns[0], 
        datos_inhibnocomp.columns[i], 
        kind='scatter',
        color=colores[i-1],
        alpha=0.8,
        s=4, 
        ax=ax,
        label=datos_inhibnocomp.columns[i])

# Además, para tener una mayor claridad acerca del efecto que tiene este tipo de
# inhibidores, se grafica la curva de Michaelis-Menten sin inhibidor   
ax.plot(
    datos_x_curva_sat, 
    resultados_ajuste_curva_sat["prediccionesModelo"], 
    color="r", 
    alpha=0.8,
    lw=2.0, 
    ls="--", 
    label="Ajuste a MM sin inhibidores")

plt.title("Título del Gráfico")     # Modificar de acuerdo a lo graficado
plt.ylabel("Nombre de eje Y")       # Modificar de acuerdo a lo graficado
plt.xlabel("Nombre de eje X")       # Modificar de acuerdo a lo graficado

# Opciones para ajustar la distribución de objetos en el gráfico
# No modificar a menos que desee experimentar!
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.tight_layout()

In [ ]:
# Ajustar curvas

# Datos deben ser transformados a arreglos numpy para ser ajustados
# IMPORTANTE: Para cambiar de columna se debe cambiar el índice de datos_y
# (donde dice datos_inhibcomp.columns[indice])
datos_x_inhibnocomp = datos_inhibnocomp[datos_inhibnocomp.columns[0]].to_numpy()
datos_y_inhibnocomp = datos_inhibnocomp[datos_inhibnocomp.columns[2]].to_numpy()

# Se ajustan los datos experimentales a los predecidos por el modelo
resultados_ajuste = ce.ajustar_modelo(ce.fit_micment_mixed_inhibitor, datos_x_inhibnocomp, datos_y_inhibnocomp)

# resultados_ajuste es un diccionario que tiene dos llaves:
# "prediccionesModelo": la cual corresponde a un arreglo de datos generados por el modelo
# "difDatosModelo": la cual corresponde a un arreglo de datos y se utilizará para graficar los residuales

In [ ]:
# Veamos qué tan bien se ajusta el modelo a los datos experimentales
fig, ax = plt.subplots(figsize=(10,5), dpi=150)

# Primero se graficarán los datos experimentales
# (recuerde que el objeto ax será donde se grafiquen las curvas)
datos_inhibnocomp.plot(
        datos_inhibnocomp.columns[0], 
        datos_inhibnocomp.columns[3], 
        kind='scatter',
        color="r",
        alpha=0.4,
        s=4, 
        ax=ax,
        label=datos_inhibnocomp.columns[3])

# Luego se graficarán los datos obtenidos mediante el modelo
ax.plot(
    datos_x_inhibnocomp, 
    resultados_ajuste["prediccionesModelo"], 
    color="b", 
    alpha=1.0,
    lw=2.0, 
    ls="--", 
    label="Ajuste")

# Se grafica ademas el ajuste a la curva de MM sin inhibidores
ax.plot(
    datos_x_curva_sat, 
    resultados_ajuste_curva_sat["prediccionesModelo"], 
    color="g", 
    alpha=0.8,
    lw=2.0, 
    ls="--", 
    label="Ajuste a MM sin inhibidores")

plt.legend()
plt.title("Título del Gráfico")     # Modificar de acuerdo a lo graficado
plt.ylabel("Nombre de eje Y")       # Modificar de acuerdo a lo graficado
plt.xlabel("Nombre de eje X")       # Modificar de acuerdo a lo graficado

## **Gráficos y Ajuste a Lineweaver-Burk**

In [ ]:
# Se crea la figura y un objeto ax
fig, ax = plt.subplots(figsize=(10,5), dpi=150)

# Se grafican los datos transformados a Lineweaver-Burk
# Recuerde que:
#     Para graficar Lineweaver-Burk deberá calcular el inverso de su set de datos
#     Debe seleccionar la columna correcta a la cual ajustó los datos anteriormente
plt.scatter(
    np.divide(1, datos_inhibnocomp[datos_inhibnocomp.columns[0]][100:]), 
    np.divide(1, datos_inhibnocomp[datos_inhibnocomp.columns[3]][100:]),
    color="b",
    alpha=0.8,
    s=4,
    label=datos_inhibnocomp.columns[3])

plt.title("Título del Gráfico")     # Modificar de acuerdo a lo graficado
plt.ylabel("Nombre de eje Y")       # Modificar de acuerdo a lo graficado
plt.xlabel("Nombre de eje X")       # Modificar de acuerdo a lo graficado

# Opciones para ajustar la distribución de objetos en el gráfico
# No modificar a menos que desee experimentar!
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.tight_layout()

In [ ]:
# Ajustar curvas

# Datos deben ser transformados a arreglos numpy para ser ajustados
# IMPORTANTE: Para cambiar de columna se debe cambiar el índice de datos_y
# (donde dice datos_inhibnocomp.columns[indice])
datos_x_lwburk = np.divide(1, datos_inhibnocomp[datos_inhibnocomp.columns[0]][100:])
datos_y_lwburk = np.divide(1, datos_inhibnocomp[datos_inhibnocomp.columns[3]][100:])

# Se ajustan los datos experimentales a los predichos por el modelo
# para mayor informacion acerca del modelo utilizado refierase a las clases de cinética enzimática
resultados_ajuste_lwburk = ce.ajustar_modelo(ce.fit_lwburk, datos_x_lwburk, datos_y_lwburk)

# resultados_ajuste es un diccionario que tiene dos llaves:
# "prediccionesModelo": la cual corresponde a un arreglo de datos generados por el modelo
# "difDatosModelo": la cual corresponde a un arreglo de datos y se utilizará para graficar los residuales

In [ ]:
# Se crea la figura y un objeto ax
fig, ax = plt.subplots(figsize=(10,5), dpi=150)

# Se grafican los datos transformados a Lineweaver-Burk
plt.scatter(
    datos_x_lwburk, 
    datos_y_lwburk,
    color="r",
    alpha=0.8,
    s=4,
    label="Datos experimentales")

# Se grafica el ajuste lineal hecho a estos datos
ax.plot(
    datos_x_lwburk, 
    resultados_ajuste_lwburk["prediccionesModelo"], 
    color="b", 
    alpha=0.8,
    lw=2.0, 
    ls="--", 
    label="Ajuste a Lineweaver-Burk")

plt.title("Título del Gráfico")     # Modificar de acuerdo a lo graficado
plt.ylabel("Nombre de eje Y")       # Modificar de acuerdo a lo graficado
plt.xlabel("Nombre de eje X")       # Modificar de acuerdo a lo graficado

# Opciones para ajustar la distribución de objetos en el gráfico
# No modificar a menos que desee experimentar!
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.tight_layout()

In [ ]:
# Esta funcion permite calcular el intercepto del eje X
ce.find_x_intercept()

## **Preguntas a desarrollar**

Complete la siguiente tabla con los valores de $V_{max}$ y $K_m$ que obtuvo en los ajustes realizados de esta sección. Recuerde que:

- Los parámetros ajustados sin inhibidor se obtuvieron en la primera sección (Curva de Saturación)

- Para obtener los parámetros a partir de Lineweaver-Burk deberá realizar cálculos (ya que no se obtienen directamente desde el ajuste a la recta)

Haz doble clic (o ingresa) para editar

|  | $V_{max}$ | $K_m$ |
|:-:|:-:|:-:|
| **Michaelis-<br>Menten** | -------- | --------- |
| Sin Inhibidor | Su valor acá | Su valor acá |
| Con Inhibidor | Su valor acá | Su valor acá |
| **Lineweaver-<br>Burk** | -------- | --------- |
| Sin Inhibidor | Su valor acá | Su valor acá |
| Con Inhibidor | Su valor acá | Su valor acá |

**En base a la tabla superior ¿Qué parámetro(s) se ve(n) afectado(s)? ¿Cómo actúan los inhibidores mixtos a nivel biológico?** (Hint: Considere efectos en la Enzima y/o complejo Enzima/Sustrato)

**La inhibición no-competitiva es considerada un caso especial de inhibición mixta. Considerando que la inhibición mixta depende (entre otros factores) de los valores de $K_{ic}$ y $K_{iu}$ ¿Qué condiciones deben cumplir estas constantes para que frente a un caso de inhibición no-competitiva?** (Hint: Recuerde que $K_{ic}$ y $K_{iu}$ son constantes de afinidad)

## **Efecto de un inhibidor acompetitivo**

In [ ]:
# Gráfico Inhibidores acompetitivos
# Recomiendo no cambiar parametros de esta funcion
ce.interactive_uncomp_inhibitor_graph1(np.linspace(0, 10000, 10000//10), ce.fit_micment_uncomp_inhibitor)

In [ ]:
# Gráfico Inhibidores acompetitivos
# Recomiendo no cambiar parametros de esta funcion
ce.interactive_uncomp_inhibitor_graph2(np.linspace(1000, 10000, 10000//10), ce.fit_micment_uncomp_inhibitor)

In [ ]:
# En primer lugar se debe leer el archivo que contiene los datos
ruta_de_archivo = "uncomp_inhibitor_data_problem_data.csv"
datos_inhibacomp = pd.read_csv(ruta_de_archivo)

datos_inhibacomp
#print(datos)

In [ ]:
colores = plt.cm.plasma_r(np.linspace(0.25, 0.75, len(list(datos_inhibacomp.columns))-1))

# Se crea la figura y un objeto ax
fig, ax = plt.subplots(figsize=(10,5), dpi=150)

# Se realiza un bucle para graficar todos los datos en una misma figura
for i in range(1, len(list(datos_inhibacomp.columns))):
    datos_inhibacomp.plot(
        datos_inhibacomp.columns[0], 
        datos_inhibacomp.columns[i], 
        kind='scatter',
        color=colores[i-1],
        alpha=0.8,
        s=4, 
        ax=ax,
        label=datos_inhibacomp.columns[i])

# Se grafica adicionalmente el ajuste inicial a la curva de MM sin inhibidores    
ax.plot(
    datos_x_curva_sat, 
    resultados_ajuste_curva_sat["prediccionesModelo"], 
    color="g", 
    alpha=0.8,
    lw=2.0, 
    ls="--", 
    label="Ajuste a MM sin inhibidores")

plt.title("Título del Gráfico")     # Modificar de acuerdo a lo graficado
plt.ylabel("Nombre de eje Y")       # Modificar de acuerdo a lo graficado
plt.xlabel("Nombre de eje X")       # Modificar de acuerdo a lo graficado

# Opciones para ajustar la distribución de objetos en el gráfico
# No modificar a menos que desee experimentar!
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.tight_layout()

In [ ]:
# Ajustar curvas

# Datos deben ser transformados a arreglos numpy para ser ajustados
# IMPORTANTE: Para cambiar de columna se debe cambiar el índice de datos_y
datos_x_inhibacomp = datos_inhibacomp[datos_inhibacomp.columns[0]].to_numpy()
datos_y_inhibacomp = datos_inhibacomp[datos_inhibacomp.columns[2]].to_numpy()

# Se ajustan los datos experimentales a los predecidos por el modelo
resultados_ajuste = ce.ajustar_modelo(ce.fit_micment_uncomp_inhibitor, datos_x_inhibacomp, datos_y_inhibacomp)

# resultados_ajuste es un diccionario que tiene dos llaves:
# "prediccionesModelo": la cual corresponde a un arreglo de datos generados por el modelo
# "difDatosModelo": la cual corresponde a un arreglo de datos y se utilizará para graficar los residuales

In [ ]:
# Veamos qué tan bien se ajusta el modelo a los datos experimentales
fig, ax = plt.subplots(figsize=(10,5), dpi=150)

# Primero se graficarán los datos experimentales
# Recuerde seleccionar la columna correspondientes a los datos elegidos
datos_inhibacomp.plot(
        datos_inhibacomp.columns[0], 
        datos_inhibacomp.columns[2], 
        kind='scatter',
        color="r",
        alpha=0.4,
        s=4, 
        ax=ax,
        label=datos_inhibacomp.columns[2])

# Luego se graficarán los datos obtenidos mediante el modelo
ax.plot(
    datos_x_inhibacomp, 
    resultados_ajuste["prediccionesModelo"], 
    color="b", 
    alpha=1.0,
    lw=2.0, 
    ls="--", 
    label="Ajuste")

# Se grafica ademas el ajuste a la curva de MM sin inhibidores
ax.plot(
    datos_x_curva_sat, 
    resultados_ajuste_curva_sat["prediccionesModelo"], 
    color="g", 
    alpha=0.8,
    lw=2.0, 
    ls="--", 
    label="Ajuste a MM sin inhibidores")

plt.legend()
plt.title("Título del Gráfico")     # Modificar de acuerdo a lo graficado
plt.ylabel("Nombre de eje Y")       # Modificar de acuerdo a lo graficado
plt.xlabel("Nombre de eje X")       # Modificar de acuerdo a lo graficado

## **Gráficos y Ajuste a Lineweaver-Burk**

In [ ]:
# Se crea la figura y un objeto ax
fig, ax = plt.subplots(figsize=(10,5), dpi=150)

# Se realiza un bucle para graficar todos los datos en una misma figura
plt.scatter(
    np.divide(1, datos_inhibacomp[datos_inhibacomp.columns[0]][200:]), 
    np.divide(1, datos_inhibacomp[datos_inhibacomp.columns[2]][200:]),
    color="b",
    alpha=0.8,
    s=4,
    label=datos_inhibacomp.columns[2])

plt.title("Título del Gráfico")     # Modificar de acuerdo a lo graficado
plt.ylabel("Nombre de eje Y")       # Modificar de acuerdo a lo graficado
plt.xlabel("Nombre de eje X")       # Modificar de acuerdo a lo graficado

# Opciones para ajustar la distribución de objetos en el gráfico
# No modificar a menos que desee experimentar!
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.tight_layout()

In [ ]:
# Ajustar curvas

# Datos deben ser transformados a arreglos numpy para ser ajustados
# IMPORTANTE: Para cambiar de columna se debe cambiar el índice de datos_y
# (donde dice datos_inhibacomp.columns[índice])
datos_x_lwburk = np.divide(1, datos_inhibacomp[datos_inhibacomp.columns[0]][200:])
datos_y_lwburk = np.divide(1, datos_inhibacomp[datos_inhibacomp.columns[2]][200:])

# Se ajustan los datos experimentales a los predichos por el modelo
# para mayor informacion acerca del modelo utilizado refierase a las clases de cinética enzimática
resultados_ajuste_lwburk = ce.ajustar_modelo(ce.fit_lwburk, datos_x_lwburk, datos_y_lwburk)

# resultados_ajuste es un diccionario que tiene dos llaves:
# "prediccionesModelo": la cual corresponde a un arreglo de datos generados por el modelo
# "difDatosModelo": la cual corresponde a un arreglo de datos y se utilizará para graficar los residuales

In [ ]:
# Se crea la figura y un objeto ax
fig, ax = plt.subplots(figsize=(10,5), dpi=150)

# Se grafican los datos transformados a Lineweaver-Burk
plt.scatter(
    datos_x_lwburk, 
    datos_y_lwburk,
    color="r",
    alpha=0.8,
    s=4,
    label="Datos experimentales")

# Se grafica también la recta obtenida luego de ajustar los datos al modelo
ax.plot(
    datos_x_lwburk, 
    resultados_ajuste_lwburk["prediccionesModelo"], 
    color="b", 
    alpha=0.8,
    lw=2.0, 
    ls="--", 
    label="Ajuste a Lineweaver-Burk")

plt.title("Título del Gráfico")     # Modificar de acuerdo a lo graficado
plt.ylabel("Nombre de eje Y")       # Modificar de acuerdo a lo graficado
plt.xlabel("Nombre de eje X")       # Modificar de acuerdo a lo graficado

# Opciones para ajustar la distribución de objetos en el gráfico
# No modificar a menos que desee experimentar!
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.tight_layout()

In [ ]:
# Esta funcion permite calcular el intercepto del eje X
ce.find_x_intercept()

##**Preguntas a desarrollar**

Complete la siguiente tabla con los valores de $V_{max}$ y $K_m$ que obtuvo en los ajustes realizados de esta sección. Recuerde que:

- Los parámetros ajustados sin inhibidor se obtuvieron en la primera sección (Curva de Saturación)

- Para obtener los parámetros a partir de Lineweaver-Burk deberá realizar cálculos (ya que no se obtienen directamente desde el ajuste a la recta)

Haz doble clic (o ingresa) para editar

|  | $V_{max}$ | $K_m$ |
|:-:|:-:|:-:|
| **Michaelis-<br>Menten** | -------- | --------- |
| Sin Inhibidor | Su valor acá | Su valor acá |
| Con Inhibidor | Su valor acá | Su valor acá |
| **Lineweaver-<br>Burk** | -------- | --------- |
| Sin Inhibidor | Su valor acá | Su valor acá |
| Con Inhibidor | Su valor acá | Su valor acá |

**En base a la tabla superior ¿Qué parámetro(s) se ve(n) afectado(s)? ¿Cómo actúan los inhibidores acompetitivos/uncomptetitivos a nivel biológico?** (Hint: Considere efectos en la Enzima y/o complejo Enzima/Sustrato)

**¿Por qué en la inhibición acompetitiva/uncompetitiva se obtienen rectas paralelas en el gráfico de Lineweaver-Burk?**

## **Inhibidor Incognito**

###Usted contará con un archivo llamado "incognito.csv" en su carpeta de Google Drive, el cual tiene **seis** columnas: Concentracion de Sutrato, Tasa de Reacción y 4 columnas de Tasas de Reacción (con inhibidor) a diferentes concentraciones del inhibidor. A partir de estos datos, deberá:

#### ***Elegir una concentración de inhibidor para hacer las actividades***.
- Construir un gráfico con la curva de **Michaelis-Menten**.
- **Ajustar** los datos al modelo de Michaelis-Menten.
- Graficar los datos experimentales (recuerde modificarlos para Lineweaver-Burk) junto a los datos predichos por el ajuste.

- Construir un gráfico de **Lineweaver-Burk**.
- **Ajustar** los datos al modelo de Lineweaver-Burk.
- Graficar los datos experimentales (recuerde modificarlos para Lineweaver-Burk) junto a los datos predichos por el ajuste.

- Explicar a partir de los parámetros ajustados a **qué tipo de inhibidor corresponde** el que le fue asignado.

### Esta sección debe desarrollarla en base a los que ha sido explicado en los cuadernillos de:
- ### Introducción a Python
- ### Cinética Enzimática I
- ### Cinética Eznimática II

### Recuerde que al comienzo de esta sección se mencionó que no era necesario reinventar la rueda; **use el código de estos cuadernillos, y si prefiere cambiar parte del código, adelante!** Todo lo necesario para desarrollar sus actividades se encuentra en los tres cuadernillos que tiene a su disposición.

In [ ]:
# Para su código




In [ ]:
# Para su código

In [ ]:
# Para su código

In [ ]:
# Para su código

In [ ]:
# Para su código

In [ ]:
# Para su código

In [ ]:
# Para su código

In [ ]:
# Para su código
# (Si lo requiere puede agregar más celdas abajo)

## **Actividad**

**Una vez hecho su código, los gráficos y los ajustes correspondientes, complete la siguiente tabla y a continuación desarrolle la pregunta**

Haz doble clic (o ingresa) para editar

|  | $V_{max}$ | $K_m$ |
|:-:|:-:|:-:|
| **Michaelis-<br>Menten** | -------- | --------- |
| Sin Inhibidor | Su valor acá | Su valor acá |
| Con Inhibidor | Su valor acá | Su valor acá |
| **Lineweaver-<br>Burk** | -------- | --------- |
| Sin Inhibidor | Su valor acá | Su valor acá |
| Con Inhibidor | Su valor acá | Su valor acá |


- **¿A qué tipo de inhibidor corresponde el que le fue asignado? Fundamente en base a los parámetros ajustados.**

Escriba su respuesta acá

## **Exportar cuadernillo a PDF**

### IMPORTANTE: Debido a que el proceso de instalacion es remoto, la ejecución de la primera celda tarda entre 10-15 minutos. (Se recomienda ejecutarla una vez resuelto el cuadernillo y cuando se quiera convertir a PDF)

### En caso de obtener errores al convertir el cuadernillo a PDF se recomienda lo siguiente: **Ir a Entorno de ejecución -> Restablecer la configuracion de fabrica del entorno** (no se preocupen ya que el código **no cambia**; piensen de esto como un formateo al sistema operativo). Luego de hacer esto, volver a ejecutar la celda de abajo para una instalación nueva de LaTeX y la última celda para exportar el cuadernillo.

**Si esta celda le trae muchos problemas, no la ejecute y guarde el cuadernillo en su carpeta de trabajo (Archivo -> Guardar). Asegurese de que quedó guardado correctamente ingresando a su carpeta y viendo la última fecha y hora de modificación.**

In [ ]:
%%capture out

# Instala paquetes necesarios para renderizar LaTeX
!apt-get install texlive-full
!apt-get install pandoc
!apt-get install texlive-xetex

# Instala paquetes necesarios para convertir a PDF
!pip install metakernel
!pip install jupyterlab
!pip install jupyterlab_latex
!pip install nbconvert

import nbconvert

In [ ]:
# Instala extensiones para widgets interactivos
!pip2.7 install https://github.com/ipython-contrib/jupyter_contrib_nbextensions/tarball/master
!jupyter contrib nbextension install --system 
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
# Importar nbconvert para convertir a PDF su cuadernillo
import nbconvert
!jupyter nbconvert "/content/drive/Shared drives/GC_Cinética - Alejandro Aravena/Cinetica_Enzimatica_1.ipynb" --to pdf